In [37]:
%pip install numpy pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\acer\appdata\local\programs\python\python312\lib\site-packages (2.0.2)




[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import numpy as np
import pandas as pd

# 1. DATA INISIALISASI 

# Pembobotan Kriteria

# a. Alternatif (Mata Kuliah Pilihan)
alternatif = {
    'A1': 'Pembelajaran Mesin',
    'A2': 'Data Science',
    'A3': 'Pengenalan Pola',
    'A4': 'Manajemen Basis Data',
    'A5': 'Keamanan Jaringan Komputer',
    'A6': 'Animasi dan Multimedia'
}

# b. Data Kriteria, Tipe, Urutan (j) dan Nilai Komparatif (Sj)
# Urutan kriteria berdasarkan Nilai Rata-Rata (tj)
data_swara_input = [
    {'Kode': 'C4', 'Nama': 'Relevansi terhadap Dunia Kerja dan Industri', 'Tipe': 1, 'tj': 5.0, 'Sj': 0.0}, # Benefit
    {'Kode': 'C2', 'Nama': 'Bidang Minat / Spesialisasi', 'Tipe': 1, 'tj': 4.5, 'Sj': 0.2}, # Benefit
    {'Kode': 'C1', 'Nama': 'Prasyarat Akademik', 'Tipe': 0, 'tj': 4.0, 'Sj': 0.3}, # Cost
    {'Kode': 'C3', 'Nama': 'Kesinambungan Kurikulum', 'Tipe': 1, 'tj': 3.5, 'Sj': 0.1}, # Benefit
    {'Kode': 'C5', 'Nama': 'Tingkat Kesulitan Materi', 'Tipe': 0, 'tj': 3.0, 'Sj': 0.4}, # Cost
]
df_swara = pd.DataFrame(data_swara_input).set_index('Kode')
kriteria_list = df_swara.index.tolist()
tipe_kriteria = df_swara['Tipe'].to_dict() 

# c. Matriks Keputusan Awal (X) 
# Urutan kolom Matriks X HARUS C1, C2, C3, C4, C5 untuk kebutuhan Normalisasi
data_x = [
    [4, 5, 5, 5, 5], # A1: Pembelajaran Mesin
    [3, 4, 4, 5, 4], # A2: Data Science
    [4, 4, 3, 4, 3], # A3: Pengenalan Pola
    [2, 3, 5, 4, 2], # A4: Manajemen Basis Data
    [3, 5, 4, 4, 3], # A5: Keamanan Jaringan Komputer
    [1, 3, 3, 3, 2]  # A6: Animasi dan Multimedia
]

# Membuat DataFrame Matriks Keputusan (X) dengan urutan kolom C1, C2, C3, C4, C5
df_X_input = pd.DataFrame(data_x, index=alternatif.values(), columns=['C1', 'C2', 'C3', 'C4', 'C5'])

print("## Matriks Keputusan Awal (X):")
print(df_X_input)

## Matriks Keputusan Awal (X):
                            C1  C2  C3  C4  C5
Pembelajaran Mesin           4   5   5   5   5
Data Science                 3   4   4   5   4
Pengenalan Pola              4   4   3   4   3
Manajemen Basis Data         2   3   5   4   2
Keamanan Jaringan Komputer   3   5   4   4   3
Animasi dan Multimedia       1   3   3   3   2


In [39]:
# 2. PERHITUNGAN METODE SWARA (Bobot Wj)

# 2.1 Hitung Koefisien Perbandingan (Kj)
# Rumus: Kj = Sj + 1. Jika j=1, Kj = 1
Kj = []
for j, Sj in enumerate(df_swara['Sj']):
    # j adalah index (0, 1, 2, ...), j+1 adalah Urutan (1, 2, 3, ...)
    if (j + 1) == 1:
        Kj.append(1.0) # Urutan 1, Kj = 1
    else:
        Kj.append(Sj + 1)
df_swara['Kj (Koefisien Perbandingan)'] = Kj

# 2.2 Hitung Koefisien Berurutan (Qj)
# Rumus: Qj = Qj-1 / Kj. Jika j=1, Qj = 1
Qj = []
for j, Kj_val in enumerate(df_swara['Kj (Koefisien Perbandingan)']):
    if (j + 1) == 1:
        Qj.append(1.0) # Urutan 1, Qj = 1
    else:
        # Qj = Qj-1 / Kj
        Qj.append(Qj[-1] / Kj_val) # Qj[-1] adalah Qj-1
df_swara['Qj (Koefisien Berurutan)'] = Qj

# 2.3 Hitung Bobot Akhir (Wj)
# Rumus: Wj = Qj / Sigma(Qj)
Sigma_Qj = sum(Qj)
Wj = [q / Sigma_Qj for q in Qj]
df_swara['Wj (Bobot Akhir)'] = Wj

# Tampilkan hasil perhitungan SWARA (untuk verifikasi dengan Gambar b)
pd.options.display.float_format = '{:.10f}'.format 
print("## Hasil Perhitungan SWARA (Bobot Kriteria):")
df_swara[['Sj', 'Kj (Koefisien Perbandingan)', 'Qj (Koefisien Berurutan)', 'Wj (Bobot Akhir)']]

## Hasil Perhitungan SWARA (Bobot Kriteria):


,Sj,Kj (Koefisien Perbandingan),Qj (Koefisien Berurutan),Wj (Bobot Akhir)
Kode,,,,
C4,0.0000000000,1.0000000000,1.0000000000,0.2879056613
C2,0.2000000000,1.2000000000,0.8333333333,0.2399213844
C1,0.3000000000,1.3000000000,0.6410256410,0.1845549111
C3,0.1000000000,1.1000000000,0.5827505828,0.1677771919
C5,0.4000000000,1.4000000000,0.4162504163,0.1198408513


In [40]:
# Ambil bobot Wj untuk setiap kriteria dalam urutan C1, C2, C3, C4, C5 (diperlukan untuk perkalian Matriks X)
# Ubah urutan df_swara dari (C4, C2, C1, C3, C5) ke (C1, C2, C3, C4, C5)
bobot_swara = df_swara['Wj (Bobot Akhir)'].reindex(['C1', 'C2', 'C3', 'C4', 'C5']).values

print(f"\nBobot Akhir (Wj) dalam urutan [C1, C2, C3, C4, C5]: {bobot_swara.round(10)}")


Bobot Akhir (Wj) dalam urutan [C1, C2, C3, C4, C5]: [0.18455491 0.23992138 0.16777719 0.28790566 0.11984085]


In [41]:
# 3. PERHITUNGAN METODE SAW (NORMALISASI DAN PREFERENSI)

# a. Normalisasi Matriks (R)
df_R = df_X_input.copy() 

max_vals = df_X_input.max()
min_vals = df_X_input.min()

for j in df_X_input.columns:
    tipe = tipe_kriteria[j] # Ambil Tipe (1=Benefit, 0=Cost)
    
    if tipe == 1:
        # Benefit: rij = xij / max(xij)
        df_R[j] = df_X_input[j] / max_vals[j]
    else:
        # Cost: rij = min(xij) / xij
        df_R[j] = min_vals[j] / df_X_input[j]

print("Matriks Normalisasi (R):")
df_R

Matriks Normalisasi (R):


,C1,C2,C3,C4,C5
Pembelajaran Mesin,0.2500000000,1.0000000000,1.0000000000,1.0000000000,0.4000000000
Data Science,0.3333333333,0.8000000000,0.8000000000,1.0000000000,0.5000000000
Pengenalan Pola,0.2500000000,0.8000000000,0.6000000000,0.8000000000,0.6666666667
Manajemen Basis Data,0.5000000000,0.6000000000,1.0000000000,0.8000000000,1.0000000000
Keamanan Jaringan Komputer,0.3333333333,1.0000000000,0.8000000000,0.8000000000,0.6666666667
Animasi dan Multimedia,1.0000000000,0.6000000000,0.6000000000,0.6000000000,1.0000000000


In [42]:
# b. Perhitungan Nilai Preferensi (Vi) dan Perankingan
# Rumus: Vi = Sigma (Wj * rij)
R_matrix = df_R.values
W_vector = bobot_swara # Menggunakan bobot hasil SWARA (C1, C2, C3, C4, C5)

# Hitung Nilai Preferensi (Vi)
V_scores = R_matrix.dot(W_vector)

# Membuat DataFrame Hasil Akhir
results = pd.DataFrame({
    'Alternatif': alternatif.values(),
    'Nilai Preferensi (Vi)': V_scores
})

# Lakukan Perankingan (Peringkat 1 = Vi tertinggi)
results['Peringkat'] = results['Nilai Preferensi (Vi)'].rank(ascending=False, method='min').astype(int)

# Urutkan dan Tampilkan Hasil
df_final = results.sort_values(by='Nilai Preferensi (Vi)', ascending=False).reset_index(drop=True)

print("## Hasil Akhir Perankingan (SWARA-SAW):")
df_final

## Hasil Akhir Perankingan (SWARA-SAW):


,Alternatif,Nilai Preferensi (Vi),Peringkat
0,Pembelajaran Mesin,0.7896793059,1
1,Manajemen Basis Data,0.7541728584,2
2,Keamanan Jaringan Komputer,0.7458798715,3
3,Data Science,0.7355032517,4
4,Animasi dan Multimedia,0.7217583050,5
5,Pengenalan Pola,0.6489605803,6


In [45]:
print("Kesimpulan:")
print(f"Keputusan Terbaik terbaik untuk pemilihan mata kuliah semester 5 di Fakultas Ilmu Komputer Teknik Informatika adalah {df_final.iloc[0]['Alternatif']}")

Kesimpulan:
Keputusan Terbaik terbaik untuk pemilihan mata kuliah semester 5 di Fakultas Ilmu Komputer Teknik Informatika adalah Pembelajaran Mesin
